<a href="https://colab.research.google.com/github/Vidit122/genAiRP/blob/complete/CompleteRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:00


In [ ]:
def char_to_binary(char):
    if char.isalpha():
        char_value = ord(char.lower()) - ord('a') + 1
        return bin(char_value)[2:].zfill(5)
    else:
        return None

def string_to_binary(string):
    binary_string = ""
    for char in string:
        binary_char = char_to_binary(char)
        if binary_char:
            binary_string += binary_char
    return binary_string

# Example usage:
input_string = input("Enter a string: ")
binary_value = string_to_binary(input_string)
print("Binary value:", binary_value)

Enter a string: hi
Binary value: 0100001001


In [ ]:
import random

def pad_string_to_multiple_of_4(input_string):
    remainder = len(input_string) % 4
    if remainder == 0:
        return input_string
    else:
        padding_length = 4 - remainder
        padding = ''.join(str(random.randint(0, 1)) for _ in range(padding_length))
        return input_string + padding

# Example usage:
padded_string = pad_string_to_multiple_of_4(binary_value)
print("Padded string:", padded_string)
101000100110

Padded string: 010000100110


101000100110

In [ ]:
feature_map={
    "place":{'00': 'mountain','01': 'sea','10': 'building','11': 'forest'},
    "animal":{'00': 'dog','10': 'spider','01': 'sheep','11': 'chicken'}
}


input_string = padded_string
maps = "place"
prompts = []
for i in range(0, len(input_string), 2):
    feature = input_string[i:i+2]
    maps = "animal" if maps == "place" else "place"
    prompts.append(feature_map[maps][feature])
print(prompts)

prompt_chain = []
for i in range(0, len(prompts), 2):
    [a,b]= prompts[i:i+2]
    prompt_chain.append(b+" behind a "+a)
print(prompt_chain)


['sheep', 'mountain', 'dog', 'building', 'sheep', 'building']
['mountain behind a sheep', 'building behind a dog', 'building behind a sheep']


### Here we load and run the models

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors"

unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:244: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all o

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

sdxl_lightning_4step_unet.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

...

tokenizer_2/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
import pickle
place_path = '/content/drive/MyDrive/ML/Models/finalized_model.pkl'
animal_path = '/content/drive/MyDrive/ML/Models/final_animal_model.pkl'


p_model = pickle.load(open(place_path,'rb'))
a_model = pickle.load(open(animal_path,'rb'))

### Here we generate the image from prompt

In [ ]:
count = 0
for prompt in prompt_chain:
  count+=1
  pipe(prompt, num_inference_steps=400, guidance_scale=0).images[0].save(f"output{count}.png")

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
import cv2
import numpy as np

def load_and_preprocess_place(image_path):
  IMAGE_SIZE = (150, 150)
  # Load the image
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, IMAGE_SIZE)

  # Normalize the image
  image = image / 255.0

  return image

def predict_place(p_model, image_path,class_names):
  # Load and preprocess the image
  image = load_and_preprocess_place(image_path)

  # Add batch dimension
  image = np.expand_dims(image, axis=0)

  # Predict the class
  predictions = p_model.predict(image)
  pred_label = np.argmax(predictions, axis=1)[0]

  return class_names[pred_label]

def load_and_preprocess_animal(image_path):
  TARGET_SIZE = (224, 224)
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, TARGET_SIZE)
  img = img.reshape(1,224,224,3)

  return img

def predict_animal(a_model,image_path,labels):
  image = load_and_preprocess_animal(image_path)
  pred = a_model.predict(image)
  pred = np.argmax(pred,axis=1)

  return labels[pred[0]]

In [ ]:
def generate_byte(p_model,a_model,image_path):
  class_names = ['mountain', 'buildings', 'sea', 'forest']
  labels= {0: 'cane', 1: 'gallina', 2: 'pecora', 3: 'ragno'}
  animal = predict_animal(a_model,image_path,labels)
  place = predict_place(p_model,image_path,class_names)
  print(animal,place)
  bit = feature_map_rev["animal"][animal] + feature_map_rev["place"][place]
  return bit

feature_map_rev={
    'place': {'mountain': '00', 'sea': '01', 'buildings': '10', 'forest': '11'},
    'animal': {'cane': '00', 'ragno': '10', 'pecora': '01', 'gallina': '11'}
}

import os
import glob

png_paths = []
current_directory = "/content"
png_paths = glob.glob(current_directory + "/*.png")
print(png_paths)
png_paths.sort()
print(png_paths)

['/content/output2.png', '/content/output3.png', '/content/output1.png']
['/content/output1.png', '/content/output2.png', '/content/output3.png']


In [ ]:
def convert_binary_to_alphabets(binary_string):
    alphabet_string = ""
    remainder = len(binary_string) % 5
    if remainder == 0:
        binary_string = binary_string
    else:
        binary_string = binary_string[:-remainder]

    for i in range(0, len(binary_string), 5):
        binary_chunk = binary_string[i:i+5]
        decimal_value = int(binary_chunk, 2)
        alphabet_character = chr(ord('a') + decimal_value - 1)
        alphabet_string += alphabet_character
    return alphabet_string

# Example usage:
input_string = "1010110001110010101001100"  # Example binary string
alphabet_string = convert_binary_to_alphabets(input_string)
print("Alphabet string:", alphabet_string)


def decode(png_paths):
  message = ""
  for img in png_paths:
    bit = generate_byte(p_model,a_model,img)
    message += bit
  print(message)
  plaintext = convert_binary_to_alphabets(message)
  return plaintext

Alphabet string: uqyjl


In [ ]:
answer = decode(png_paths)
print(answer)

1/1 [==============================] - 0s 20ms/step
pecora mountain
1/1 [==============================] - 0s 20ms/step
cane buildings
1/1 [==============================] - 0s 28ms/step
pecora buildings
010000100110
hi
